In [1]:
#Data preprocessing libraries
import pandas as pd  #Data manipulation
import string  #Remove punctuation & characters
import nltk  #Natural language processing 
import pickle  #For loading saved models and vectorizers
import re


from nltk.corpus import stopwords  #Stop word removal
from nltk.tokenize import word_tokenize  #Tokenizition
from nltk.stem import WordNetLemmatizer  #Import WordNetLemmatizer
from nltk.corpus import wordnet  #Import WordNet

#Feature extractions libraries
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#Models libraries
from sklearn.model_selection import train_test_split #For data splitting
#Model Evaluation Function
from sklearn.metrics import accuracy_score, classification_report  #Import metrics
from sklearn.svm import SVC  #SVM Model
from sklearn.naive_bayes import MultinomialNB #Naive Bayes Model -  MultinomialNB 
from sklearn.ensemble import RandomForestClassifier #Random Forest Model

#to arabic classification
import argparse
import numpy as np
import pyarabic.araby as araby
import qalsadi.lemmatizer
from collections import Counter

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#Read excel file
data = pd.read_csv(r'C:\Users\s7r_2\Downloads\bbc-text (1).csv')
data.head()

,Unnamed: 0,category,text,cleaned_text,tokens,lemmatized,lemmatized_text
0,0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"['tv', 'future', 'hands', 'viewers', 'home', '...","['tv', 'future', 'hand', 'viewer', 'home', 'th...",tv future hand viewer home theatre system plas...
1,1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"['worldcom', 'boss', 'left', 'books', 'alone',...","['worldcom', 'bos', 'leave', 'book', 'alone', ...",worldcom bos leave book alone former worldcom ...
2,2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"['tigers', 'wary', 'farrell', 'gamble', 'leice...","['tiger', 'wary', 'farrell', 'gamble', 'leices...",tiger wary farrell gamble leicester say rush m...
3,3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"['yeading', 'face', 'newcastle', 'fa', 'cup', ...","['yeading', 'face', 'newcastle', 'fa', 'cup', ...",yeading face newcastle fa cup premiership side...
4,4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"['ocean', 'twelve', 'raids', 'box', 'office', ...","['ocean', 'twelve', 'raid', 'box', 'office', '...",ocean twelve raid box office ocean twelve crim...


In [9]:
# دالة لإزالة الرموز غير ASCII
def _removeNonAscii(s): 
    return "".join(i for i in s if ord(i) < 128)

# دالة لتنظيف النصوص
def clean_text(text):
    # تأكد من أن المتغير هو نص
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = text.replace('(ap)', '')
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    text = re.sub('[^a-zA-Z ?!]+', '', text)
    text = _removeNonAscii(text)
    text = text.strip()
    return text

# تحويل قائمة الكلمات إلى نص مفرد
def list_to_string(tokens):
    return ' '.join(tokens)

# تأكد من وجود عمود 'tokens' كقوائم
# تطبيق دالة تحويل القوائم إلى نصوص ثم دالة التنظيف على عمود tokens
data['cleaned_text'] = data['text'].apply(clean_text)

# عرض أول خمس صفوف من DataFrame بعد التنظيف
data.head()

,Unnamed: 0,category,text,cleaned_text,tokens,lemmatized,lemmatized_text
0,0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"['tv', 'future', 'hands', 'viewers', 'home', '...","['tv', 'future', 'hand', 'viewer', 'home', 'th...",tv future hand viewer home theatre system plas...
1,1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"['worldcom', 'boss', 'left', 'books', 'alone',...","['worldcom', 'bos', 'leave', 'book', 'alone', ...",worldcom bos leave book alone former worldcom ...
2,2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"['tigers', 'wary', 'farrell', 'gamble', 'leice...","['tiger', 'wary', 'farrell', 'gamble', 'leices...",tiger wary farrell gamble leicester say rush m...
3,3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"['yeading', 'face', 'newcastle', 'fa', 'cup', ...","['yeading', 'face', 'newcastle', 'fa', 'cup', ...",yeading face newcastle fa cup premiership side...
4,4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"['ocean', 'twelve', 'raids', 'box', 'office', ...","['ocean', 'twelve', 'raid', 'box', 'office', '...",ocean twelve raid box office ocean twelve crim...


In [10]:
# تحميل قائمة الكلمات الشائعة باللغة الإنجليزية
stop_words = set(stopwords.words('english'))

# دالة لتوكنة النصوص وإزالة الكلمات الشائعة
def tokenize_and_remove_stopwords(text):
    if not isinstance(text, str):  # تحقق من أن المدخل هو نص
        return []
    
    # توكنة النص
    tokens = nltk.word_tokenize(text)
    # إزالة الكلمات الشائعة
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    return filtered_tokens

# تطبيق دالة توكنة النصوص وإزالة الكلمات الشائعة على عمود 'cleaned_text'
data['tokens'] = data['cleaned_text'].apply(tokenize_and_remove_stopwords)
# عرض أول خمس صفوف من DataFrame بعد معالجة النصوص
data.head()

,Unnamed: 0,category,text,cleaned_text,tokens,lemmatized,lemmatized_text
0,0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"[tv, future, hands, viewers, home, theatre, sy...","['tv', 'future', 'hand', 'viewer', 'home', 'th...",tv future hand viewer home theatre system plas...
1,1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"[worldcom, boss, left, books, alone, former, w...","['worldcom', 'bos', 'leave', 'book', 'alone', ...",worldcom bos leave book alone former worldcom ...
2,2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"[tigers, wary, farrell, gamble, leicester, say...","['tiger', 'wary', 'farrell', 'gamble', 'leices...",tiger wary farrell gamble leicester say rush m...
3,3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"[yeading, face, newcastle, fa, cup, premiershi...","['yeading', 'face', 'newcastle', 'fa', 'cup', ...",yeading face newcastle fa cup premiership side...
4,4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"[ocean, twelve, raids, box, office, ocean, twe...","['ocean', 'twelve', 'raid', 'box', 'office', '...",ocean twelve raid box office ocean twelve crim...


In [22]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# تحميل wordnet وملفات التصنيف النحوي
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# تهيئة WordNetLemmatizer
wn = WordNetLemmatizer()

# دالة لتحويل التصنيف النحوي من NLTK إلى WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # الافتراضي هو الاسم

# دالة التجذير (lemmatizing) مع التصنيف النحوي
def lemmatizing(tokens):
    # تصنيف الكلمات النحوي
    pos_tags = nltk.pos_tag(tokens)
    # تطبيق التجذير باستخدام التصنيف النحوي
    return [wn.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]

# تطبيق دالة lemmatizing على عمود 'tokens'
data['lemmatized'] = data['tokens'].apply(lemmatizing)

# عرض أول خمس صفوف للتحقق من النتائج
data.head()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,Unnamed: 0,category,text,cleaned_text,tokens,lemmatized,lemmatized_text
0,0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"[tv, future, hands, viewers, home, theatre, sy...","[tv, future, hand, viewer, home, theatre, syst...","[ ' t v ' , ' f u t u r e ' , ' h a n d ' ..."
1,1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"[worldcom, boss, left, books, alone, former, w...","[worldcom, bos, leave, book, alone, former, wo...","[ ' w o r l d c o m ' , ' b o s ' , ' l e ..."
2,2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"[tigers, wary, farrell, gamble, leicester, say...","[tiger, wary, farrell, gamble, leicester, say,...","[ ' t i g e r ' , ' w a r y ' , ' f a r r ..."
3,3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"[yeading, face, newcastle, fa, cup, premiershi...","[yeading, face, newcastle, fa, cup, premiershi...","[ ' y e a d i n g ' , ' f a c e ' , ' n e ..."
4,4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"[ocean, twelve, raids, box, office, ocean, twe...","[ocean, twelve, raid, box, office, ocean, twel...","[ ' o c e a n ' , ' t w e l v e ' , ' r a ..."


In [11]:
data['lemmatized_text'] = data['lemmatized'].apply(lambda x: ' '.join(x)) #To convert the texts in the “lemmatized” column into single texts by joining the words using spaces.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer  # تأكد من استيراد CountVectorizer

#Bag of Words (BoW)
bow_vectorizer = CountVectorizer(min_df=5)  # يمكنك تجربة قيم أعلى حسب حجم البيانات
bow_matrix = bow_vectorizer.fit_transform(data['lemmatized_text'])
bow_df = pd.DataFrame.sparse.from_spmatrix(bow_matrix, columns=bow_vectorizer.get_feature_names_out())
print(bow_df)


      aaa  aaas  aaron  abandon  abandonment  abbott  abc  abide  ability  \
0       0     0      0        0            0       0    0      1        0   
1       0     0      0        0            0       0    0      0        1   
2       0     0      0        0            0       0    0      0        0   
3       0     0      0        0            0       0    0      0        0   
4       0     0      0        0            0       0    0      0        0   
...   ...   ...    ...      ...          ...     ...  ...    ...      ...   
2220    0     0      0        0            0       0    0      0        0   
2221    0     0      0        0            0       0    0      0        0   
2222    0     0      0        0            0       0    0      0        0   
2223    0     0      0        0            0       0    0      0        0   
2224    0     0      0        0            0       0    0      0        0   

      able  ...  zach  zealand  zeppelin  zero  zhang  zimbabwe  zombie  zo

In [5]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#TF-IDF
tfidf_BBC = TfidfVectorizer(min_df=1)#Ignore rare words
tfidf_matrix = tfidf_BBC.fit_transform(data['lemmatized_text'])#tfidf_matrix-Represents the weight of each word based on its frequency.
#Fit_transform-To convert texts to a matrix.


print("\nTF-IDF Matrix:")
print(tfidf_matrix)
with open("tfidf_BBC.pickle", "wb") as file:
    pickle.dump(tfidf_BBC, file)


TF-IDF Matrix:
  (0, 13769)	0.04129311624482706
  (0, 8035)	0.02638517393687256
  (0, 10226)	0.022999743917294615
  (0, 16949)	0.032854281535374945
  (0, 13481)	0.02973975872958771
  (0, 15230)	0.02986266308566507
  (0, 22679)	0.04129311624482706
  (0, 20828)	0.06156466137781545
  (0, 2983)	0.01944992347518207
  (0, 15124)	0.039801734316358375
  (0, 19348)	0.03159221091555085
  (0, 11339)	0.04512604622002326
  (0, 15961)	0.02992486065933649
  (0, 865)	0.024538351182000186
  (0, 8313)	0.042163634491354914
  (0, 2100)	0.026831182800885714
  (0, 3550)	0.02161195267797773
  (0, 13248)	0.03267353535191786
  (0, 17429)	0.054526303886417625
  (0, 8150)	0.039801734316358375
  (0, 17205)	0.044686991958911224
  (0, 10481)	0.046087469176965504
  (0, 6986)	0.021021571111838666
  (0, 5659)	0.06156466137781545
  (0, 6237)	0.03610721093939238
  :	:
  (2224, 3837)	0.06437510095714637
  (2224, 14117)	0.04394621653051948
  (2224, 8625)	0.03387863465842343
  (2224, 8814)	0.05126497185206189
  (2224, 117

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pickle

# إعداد البيانات للتقسيم
X = tfidf_matrix
y = data['category'] 

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)  #Split data

#Function to evaluate model performance
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)  #Make predictions
    accuracy = accuracy_score(y_test, predictions)  #Calculate accuracy
    report = classification_report(y_test, predictions)  #Generate classification report
    return accuracy, report



svm_model = SVC(kernel='linear', class_weight='balanced')  #Initialize SVM model with linear kernel and class weights
svm_model.fit(X_train, y_train)  #Train the model
svm_accuracy, svm_report = evaluate_model(svm_model, X_test, y_test)  #Evaluate SVM
print("SVM Accuracy:", svm_accuracy)  #Print accuracy
print("SVM Classification Report:\n", svm_report)  #Print classification report


nb_model = MultinomialNB()  #Initialize Naive Bayes model
nb_model.fit(X_train, y_train)  #Train the model
nb_accuracy, nb_report = evaluate_model(nb_model, X_test, y_test)  #Evaluate Naive Bayes
print("Naive Bayes Accuracy:", nb_accuracy)  #Print accuracy
print("Naive Bayes Classification Report:\n", nb_report)  #Print classification report


rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)  #Initialize Random Forest model with class weights
rf_model.fit(X_train, y_train)  #Train the model
rf_accuracy, rf_report = evaluate_model(rf_model, X_test, y_test)  #Evaluate Random Forest
print("Random Forest Accuracy:", rf_accuracy)  #Print accuracy
print("Random Forest Classification Report:\n", rf_report)  #Print classification report
with open("model_bbc.pickle", "wb") as model_file:
    pickle.dump(svm_model, model_file)

SVM Accuracy: 0.9685393258426966
SVM Classification Report:
                precision    recall  f1-score   support

     business       0.96      0.92      0.94       101
entertainment       0.98      0.99      0.98        81
     politics       0.94      0.96      0.95        83
        sport       0.98      1.00      0.99        98
         tech       0.99      0.98      0.98        82

     accuracy                           0.97       445
    macro avg       0.97      0.97      0.97       445
 weighted avg       0.97      0.97      0.97       445

Naive Bayes Accuracy: 0.9573033707865168
Naive Bayes Classification Report:
                precision    recall  f1-score   support

     business       0.94      0.94      0.94       101
entertainment       1.00      0.89      0.94        81
     politics       0.92      0.98      0.95        83
        sport       0.99      1.00      0.99        98
         tech       0.94      0.98      0.96        82

     accuracy                   

In [7]:
import pickle
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# دالة معالجة البيانات
def preprocess_data(text):
    text = text.lower()  # تحويل النص إلى حروف صغيرة
    text = re.sub(r'[^\w\s\'\"]', '', text)  # إزالة علامات الترقيم
    return text

# التأكد من عدم وجود نصوص فارغة
data = data[data['lemmatized_text'].notnull() & (data['lemmatized_text'] != '')]

# معالجة البيانات
data['lemmatized_text'] = data['lemmatized_text'].apply(preprocess_data)

# إنشاء مصفوفة TF-IDF
tfidf_vectorizer = TfidfVectorizer(min_df=5)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['lemmatized_text'])

# إعداد البيانات
X = tfidf_matrix
y = data['category']

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تدريب نموذج الانحدار اللوجستي
lg_model = LogisticRegression(max_iter=200, verbose=1)
lg_model.fit(X_train, y_train)

# حفظ النموذج وtfidf_vectorizer
with open("model_bbc.pickle", "rb") as model_file:
    svm_model = pickle.load(model_file)

with open("tfidf_BBC.pickle", "rb") as file:
    tfidf_BBC = pickle.load(file)

# دالة التنبؤ
def predict_category(text):
    cleaned_text = preprocess_data(text)  # معالجة النص
    tfidf_data = tfidf_vectorizer.transform([cleaned_text])  # تحويل النص إلى تمثيل TF-IDF
    prediction = lg_model.predict(tfidf_data)  # إجراء التنبؤ
    return prediction[0]  # إرجاع الفئة المتوقعة


# اختبار دالة التنبؤ
new_text = "video games "
predicted_category = predict_category(new_text)
print(f"The predicted category is: {predicted_category}")


The predicted category is: tech


In [8]:
text = pd.read_excel(r'C:\Users\s7r_2\Downloads\��تصنيف الاخبار العربيه - نسخة.xlsx')
text.head()

,رقم التصنيف,صنف الخبر,عنوان الخبر,clean,cleaned_stopwords,lemmatized_text,lemmatized
0,0,صحة,الجلوس أكثر من 10 ساعات يومياً يفاقم مخاطر الوفاة,الجلوس اكثر من ساعات يومياً يفاقم مخاطر الوفاه,الجلوس اكثر ساعات يوميا يفاقم مخاطر الوفاه,"[('جلوس', 'noun'), ('كثر', 'verb'), ('ساعة', '...",جلوس كثر ساعة يوم فاقم مخاطر الوفاه
1,5,رياضة,مولر: هناك طريقة واحدة لإيقاف خطورة ميسي,مولر هناك طريقه واحده لايقاف خطوره ميسي,مولر طريقه واحده لايقاف خطوره ميسي,"[('مولر', 'all'), ('طريق', 'noun'), ('واحد', '...",مولر طريق واحد لايقاف خطوره ميس
2,6,اقتصاد,"وزير مالية لبنان: ""إجراءات تقشفية استثنائية"" ب...",وزير ماليه لبنان اجراءات تقشفيه استثنائيه بموا...,وزير ماليه لبنان اجراءات تقشفيه استثنائيه بموازنه,"[('زير', 'noun'), ('مال', 'noun'), ('لبن', 'no...",زير مال لبن اجراءات تقشف استثناء موازن
3,4,فن,مروان خوري: هكذا أقضي فترة الحجر المنزلي,مروان خوري هكذا اقضي فتره الحجر المنزلي,مروان خوري اقضي فتره الحجر المنزلي,"[('مرو', 'noun'), ('خور', 'noun'), ('قضى', 've...",مرو خور قضى فتر حجر منزل
4,6,اقتصاد,مصر تبدأ حملة التطعيم بلقاح كورونا لعامة الشعب,مصر تبدا حمله التطعيم بلقاح كورونا لعامه الشعب,مصر تبدا حمله التطعيم بلقاح كورونا لعامه الشعب,"[('مصر', 'noun'), ('بد', 'verb'), ('حمل', 'nou...",مصر بد حمل تطعيم لقاح كار عام شعب


In [9]:
from sklearn.feature_extraction.text import CountVectorizer  # تأكد من استيراد CountVectorizer
text = pd.read_excel(r'C:\Users\s7r_2\Downloads\��تصنيف الاخبار العربيه - نسخة.xlsx')
#Bag of Words (BoW)
if 'lemmatized' in text.columns:
    # حساب Bag-of-Words
    bow_vectorizer = CountVectorizer(min_df=5, max_features=10000)  # تجاهل الكلمات النادرة
    bow_matrix = bow_vectorizer.fit_transform(text['lemmatized'])

#Convert to DataFrame for better visualization
print("Bag of Words (BoW) Matrix:")
print(bow_matrix)

Bag of Words (BoW) Matrix:
  (0, 3521)	1
  (0, 5882)	1
  (0, 4452)	1
  (0, 7947)	1
  (0, 5504)	1
  (0, 6609)	1
  (0, 1935)	1
  (1, 7241)	1
  (1, 5063)	1
  (1, 7640)	1
  (1, 6117)	1
  (1, 3914)	1
  (1, 7265)	1
  (2, 4429)	1
  (2, 6406)	1
  (2, 6129)	1
  (2, 299)	1
  (2, 3172)	1
  (2, 496)	1
  (2, 7184)	1
  (3, 6717)	1
  (3, 3953)	1
  (3, 5755)	1
  (3, 5521)	1
  (3, 3653)	1
  :	:
  (57783, 1114)	1
  (57783, 4973)	1
  (57783, 4528)	1
  (57783, 7894)	1
  (57783, 5389)	1
  (57784, 5832)	1
  (57784, 4489)	1
  (57784, 820)	1
  (57784, 2716)	1
  (57784, 4423)	1
  (57784, 7885)	1
  (57784, 2260)	1
  (57785, 6864)	1
  (57785, 3630)	1
  (57785, 4663)	1
  (57785, 2211)	1
  (57785, 7835)	1
  (57785, 4391)	1
  (57786, 7083)	1
  (57786, 4077)	1
  (57786, 4029)	1
  (57786, 2615)	1
  (57786, 3780)	1
  (57786, 4640)	1
  (57786, 3107)	1


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

text = pd.read_excel(r'C:\Users\s7r_2\Downloads\��تصنيف الاخبار العربيه - نسخة.xlsx')
#TF-IDF
tfidf_Arabic = TfidfVectorizer(min_df=1)#Ignore rare words
tfidf_matrix = tfidf_Arabic.fit_transform(text['lemmatized'])#tfidf_matrix-Represents the weight of each word based on its frequency.
#Fit_transform-To convert texts to a matrix.

#Convert to DataFrame to display the data more clearly

print("\nTF-IDF Matrix:")
print(tfidf_matrix)
with open("tfidf_Arabic.pkl", "wb") as tfidf_file:
    pickle.dump(tfidf_Arabic,tfidf_file)


TF-IDF Matrix:
  (0, 6561)	0.40070590003235035
  (0, 19245)	0.3569809072021846
  (0, 15645)	0.4787266780801195
  (0, 23502)	0.2757473843498377
  (0, 13349)	0.34733696028046385
  (0, 16561)	0.2875553569645421
  (0, 11274)	0.4510571072437371
  (1, 20755)	0.3232238638689826
  (1, 12022)	0.451721502828749
  (1, 17490)	0.49959939467166087
  (1, 21749)	0.38327049338141533
  (1, 14713)	0.32706039214003524
  (1, 20669)	0.43360084334307375
  (2, 20538)	0.4196464836883769
  (2, 1967)	0.4095251924467677
  (2, 10536)	0.48373853470560607
  (2, 1327)	0.3743634345025407
  (2, 17528)	0.30043158842465145
  (2, 18729)	0.32171591716558123
  (2, 13245)	0.2971106254355494
  (3, 20412)	0.35712435440574086
  (3, 11561)	0.38540036847408293
  (3, 15694)	0.3958948980251058
  (3, 16231)	0.3946192916142427
  (3, 12095)	0.4428547920110705
  :	:
  (57783, 3671)	0.27795453392227626
  (57783, 10066)	0.24074337432942144
  (57783, 16318)	0.2895854845623391
  (57783, 11748)	0.3279545476294673
  (57784, 7637)	0.56744031

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# تحميل مجموعة البيانات

# تحويل النصوص إلى ميزات عددية باستخدام TF-IDF
tfidf_Arabic = TfidfVectorizer(min_df=1)
X = tfidf_Arabic.fit_transform(text['lemmatized'])  # تحويل النصوص إلى مصفوفة TF-IDF

# إعداد البيانات للتقسيم
y = text['صنف الخبر'] 

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# دالة لتقييم أداء النموذج
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)  # توقع النتائج
    accuracy = accuracy_score(y_test, predictions)  # حساب الدقة
    report = classification_report(y_test, predictions)  # توليد تقرير التصنيف
    return accuracy, report

# نموذج SVM
svm_model_ar = SVC(kernel='linear', class_weight='balanced')  # تهيئة نموذج SVM
svm_model_ar.fit(X_train, y_train)  # تدريب النموذج
svm_accuracy, svm_report = evaluate_model(svm_model_ar, X_test, y_test)  # تقييم SVM
print("SVM Accuracy:", svm_accuracy)  # طباعة الدقة
print("SVM Classification Report:\n", svm_report)  # طباعة تقرير التصنيف

# نموذج Naive Bayes
nb_model = MultinomialNB()  # تهيئة نموذج Naive Bayes
nb_model.fit(X_train, y_train)  # تدريب النموذج
nb_accuracy, nb_report = evaluate_model(nb_model, X_test, y_test)  # تقييم Naive Bayes
print("Naive Bayes Accuracy:", nb_accuracy)  # طباعة الدقة
print("Naive Bayes Classification Report:\n", nb_report)  # طباعة تقرير التصنيف

# نموذج Random Forest
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)  # تهيئة نموذج Random Forest
rf_model.fit(X_train, y_train)  # تدريب النموذج
rf_accuracy, rf_report = evaluate_model(rf_model, X_test, y_test)  # تقييم Random Forest
print("Random Forest Accuracy:", rf_accuracy)  # طباعة الدقة
print("Random Forest Classification Report:\n", rf_report)  # طباعة تقرير التصنيف

# حفظ نموذج SVM
with open("model_Arabic.pickle", "wb") as model_file:
    pickle.dump(svm_model_ar, model_file)


SVM Accuracy: 0.9103651150718117
SVM Classification Report:
               precision    recall  f1-score   support

      اقتصاد       0.89      0.89      0.89      2113
 التكنولوجيا       0.92      0.91      0.92      1499
     السياحة       0.90      0.91      0.91      1029
       رياضة       0.98      0.96      0.97      1956
       سياسة       0.85      0.85      0.85      1481
         صحة       0.89      0.93      0.91      1511
          فن       0.92      0.91      0.91      1969

    accuracy                           0.91     11558
   macro avg       0.91      0.91      0.91     11558
weighted avg       0.91      0.91      0.91     11558

Naive Bayes Accuracy: 0.8857068697006403
Naive Bayes Classification Report:
               precision    recall  f1-score   support

      اقتصاد       0.75      0.95      0.84      2113
 التكنولوجيا       0.94      0.87      0.90      1499
     السياحة       0.95      0.71      0.81      1029
       رياضة       0.96      0.96      0.96     

In [22]:
from sklearn.linear_model import LogisticRegression
import re

# دالة معالجة البيانات
def preprocess_data(text):
    text = text.lower()  # تحويل النص إلى حروف صغيرة
    text = re.sub(r'[^\w\s\'\"]', '', text)  # إزالة علامات الترقيم
    return text

# تحميل البيانات
text = text[text['lemmatized'].notnull() & (text['lemmatized'] != '')]

# معالجة البيانات
text['lemmatized'] = text['lemmatized'].apply(preprocess_data)

# إنشاء مصفوفة TF-IDF
tfidf_vectorizer = TfidfVectorizer(min_df=5)
tfidf_matrix = tfidf_vectorizer.fit_transform(text['lemmatized'])

# إعداد البيانات
X = tfidf_matrix
y = text['صنف الخبر']

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تدريب نموذج الانحدار اللوجستي
lg_model = LogisticRegression(max_iter=200, verbose=1)
lg_model.fit(X_train, y_train)

# تخزين النموذج وTF-IDF
with open("model_Arabic.pickle", "rb") as model_file:
    svm_model_ar = pickle.load(model_file)

with open("tfidf_Arabic.pkl", "rb") as tfidf_file:
    tfidf_Arabic = pickle.load(tfidf_file)

# دالة التنبؤ
def predict_category(text):
    cleaned_text = preprocess_data(text)  # معالجة النص
    tfidf_data = tfidf_vectorizer.transform([cleaned_text])  # تحويل النص إلى تمثيل TF-IDF
    prediction = lg_model.predict(tfidf_data)  # إجراء التنبؤ
    return prediction[0]  # إرجاع الفئة المتوقعة

# نص جديد للتنبؤ بفئته
new_text = "وزير مالية لبنان:إجراءات تقشفية استثنائية بموازنة 2019"
predicted_category = predict_category(new_text)
print(f"The predicted category is: {predicted_category}")

The predicted category is: اقتصاد


In [ ]:
import os
import sys
from pathlib import Path
import tkinter as tk
from tkinter import messagebox
import joblib  # لتحميل النموذج

# تحميل النموذجين BBC و Arabic
#BBC
with open("model_bbc.pickle", "rb") as model_file:
    svm_model = pickle.load(model_file)
with open("tfidf_BBC.pickle", "rb") as file:
    tfidf_BBC = pickle.load(file)
    
#Arabic
with open("tfidf_Arabic.pkl", "rb") as tfidf_file:
    tfidf_Arabic = pickle.load(tfidf_file)
with open("model_Arabic.pickle", "rb") as model_file:
    svm_model_ar = pickle.load(model_file)

# المتغير لتحديد اللغة الحالية
current_language = "english"  # البداية باللغة الإنجليزية

# دالة التصنيف باستخدام النموذج المحمل
# دالة التصنيف باستخدام النموذج المحمل
def predict_category(text):
    if current_language == "english":
        text_features = tfidf_BBC.transform([text])
        prediction = svm_model.predict(text_features)
        category_map = {0: 'tech', 1: 'business', 2: 'sport', 3: 'entertainment', 4: 'politics'}
    else:
        text_features = tfidf_Arabic.transform([text])
        prediction = svm_model_ar.predict(text_features)
        category_map = {
            0: 'صحة',
            1: 'سياحي',
            2: 'سياسي', 
            3: 'تكنولوجيا',
            4: 'اقتصادي',
            5: 'رياضي'
        }

def predict_category(text):
    if current_language == "english":
        cleaned_text = preprocess_data(text)  # معالجة النص
        tfidf_data = tfidf_BBC.transform([cleaned_text])  # تحويل النص إلى تمثيل TF-IDF
        prediction = svm_model.predict(tfidf_data)  # إجراء التنبؤ
        category_map = {0: 'tech', 1: 'business', 2: 'sport', 3: 'entertainment', 4: 'politics'}
        return category_map.get(prediction[0])  # إرجاع الفئة المتوقعة
    else:
        cleaned_text = preprocess_data(text)  # معالجة النص
        tfidf_data = tfidf_Arabic.transform([cleaned_text])  # تحويل النص إلى تمثيل TF-IDF
        prediction = svm_model_ar.predict(tfidf_data)  # إجراء التنبؤ
        category_map = {
            0: 'صحة',
            1: 'سياحي',
            2: 'سياسي', 
            3: 'تكنولوجيا',
            4: 'اقتصادي',
            5: 'رياضي',
            6: 'فن'
        }
        return category_map.get(prediction[0])  # إرجاع الفئة المتوقعة
     

# إعداد دالة لتصنيف النص
def classify_text():
    text = entry.get()
    if not text.strip():
        messagebox.showwarning("خطأ", "الرجاء إدخال نص للتصنيف!")
        return
    
    # احصل على التصنيف
    category = predict_category(text)
    
    # تحديث النتيجة في واجهة tkinter
    result_label.config(text=f"نوع الخبر: {category}")
    result_label.update()  # لضمان التحديث الفوري في الواجهة

# إعداد دالة لمسح النص
def clear_text():
    entry.delete(0, tk.END)
    result_label.config(text="")

# دالة لتغيير اللغة إلى العربية
def set_language_arabic():
    global current_language
    current_language = "arabic"
    entry_label.config(text="أدخل النص:")
    classify_button.config(text="تصنيف")
    clear_button.config(text="حذف النص")
    result_label.config(text="")
    lang_frame.pack_forget()  # إخفاء شاشة اختيار اللغة
    main_frame.pack(fill="both", expand=True)  # عرض شاشة التصنيف

def set_language_english():
    global current_language
    current_language = "english"
    entry_label.config(text="Enter text:")
    classify_button.config(text="Classify")
    clear_button.config(text="Clear Text")
    result_label.config(text="")
    lang_frame.pack_forget()  # إخفاء شاشة اختيار اللغة
    main_frame.pack(fill="both", expand=True)  # عرض شاشة التصنيف

# إعداد النافذة الرئيسية
root = tk.Tk()
root.geometry("800x500")
root.title("News Classification")

# شاشة اختيار اللغة
lang_frame = tk.Frame(root)
lang_frame.pack(fill="both", expand=True)
arabic_button = tk.Button(lang_frame, text="عربي", font=("Arial", 16), command=set_language_arabic)
arabic_button.pack(side="left", expand=True, padx=20, pady=20)
english_button = tk.Button(lang_frame, text="English", font=("Arial", 16), command=set_language_english)
english_button.pack(side="right", expand=True, padx=20, pady=20)

# شاشة التصنيف الرئيسية
main_frame = tk.Frame(root)

entry_label = tk.Label(main_frame, text="", font=("Arial", 14))
entry_label.pack(pady=10)

entry = tk.Entry(main_frame, font=("Arial", 14), width=50)
entry.pack(pady=10)

classify_button = tk.Button(main_frame, text="", font=("Arial", 14), command=classify_text)
classify_button.pack(pady=10)

clear_button = tk.Button(main_frame, text="", font=("Arial", 14), command=clear_text)
clear_button.pack(pady=10)

result_label = tk.Label(main_frame, text="", font=("Arial", 16), fg="blue")
result_label.pack(pady=20)

# بدء تشغيل النافذة
root.mainloop()
